In [71]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import re
import urllib.request, urllib.parse

base_url = "https://www.pro-football-reference.com/schools/"

In [204]:
def check_college_qb(tr):
    if tr.name == "tr":
        if len(tr.contents[0].contents) > 0:
            return tr.contents[0].contents[0] == "QB" and len(tr.contents[2].contents) > 0
        else:
            return False
        
    return False

def get_relative(a):
    return a.get("href")
                                                  
def get_school_qb_links(school_url):
    links = []
    sauce = urllib.request.urlopen(school_url).read()
    soup = BeautifulSoup(sauce, "html.parser")
    table = soup.find("table", id="all_players").find("tbody")
    rows = table.find_all(check_college_qb)
    for row in rows:
        # Clean up lol
        elements = row.find_all("a")[:2]
        pair = tuple(map(lambda r : urllib.parse.urljoin(school_url, r), map(get_relative, elements)))
        name = elements[0].contents[0]
        links.append((pair, name))
        
    return links

def get_school_links(url):
    links = []
    sauce = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(sauce, "html.parser")
    tds = soup.find_all("td", attrs={'data-stat': "college_name"})
    for td in tds:
        link = td.find("a").get("href")
        # Find more elegant solution
        if not "none" in link:
            links.append(urllib.parse.urljoin(url, link))

    return links

def get_passing_table(url):
    df = None
    sauce = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(sauce, "html.parser")
    try: 
        df = pd.read_html(str(soup.find("table", id="passing")))[0]
    except:
        table = find_comment_table(soup)
        if table is not None:
            df = pd.read_html(table)[0]
    finally:
        return df
            
def find_comment_table(soup):
    for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find('id="passing"') > 0:
            fresh_soup = BeautifulSoup(comment, "html.parser")
            return str(fresh_soup.find("table", id="passing"))
        
    return None

def get_player_tables(player_urls, id_):
    nfl = get_passing_table(player_urls[0])
    if nfl is not None:
        nfl['player_id'] = id_
    cfl = get_passing_table(player_urls[1])
    if cfl is not None:
        cfl.columns = cfl.columns.droplevel([0])
        cfl['player_id'] = id_    
    return (nfl, cfl)

In [207]:
names = []
nfl = pd.DataFrame()
cfl = pd.DataFrame()
i = 0
for school in get_school_links(base_url):
    for qb in get_school_qb_links(school):
        names.append(qb[1])
        tables = get_player_tables(qb[0], i)
        nfl = pd.concat([nfl, tables[0]])
        cfl = pd.concat([cfl, tables[1]])
        i += 1

names_s = pd.Series(names)

('https://www.pro-football-reference.com/players/K/KnafGr20.htm', 'http://www.sports-reference.com/cfb/players/greg-knafelc-1.html')
('https://www.pro-football-reference.com/players/P/PtacBo20.htm', 'http://www.sports-reference.com/cfb/players/bob-ptacek-1.html')
('https://www.pro-football-reference.com/players/J/JoacSt20.htm', 'http://www.sports-reference.com/cfb/players/steve-joachim-1.html')
('https://www.pro-football-reference.com/players/S/StevTo01.htm', 'http://www.sports-reference.com/cfb/players/tommy-stevens-1.html')
('https://www.pro-football-reference.com/players/B/BurnEd20.htm', 'http://www.sports-reference.com/cfb/players/ed-burns-1.html')
<tr><th class="left iz" data-stat="pos" scope="row"></th><td class="left" csk="Crowder,Tae" data-append-csv="CrowTa00" data-stat="player"><a href="/players/C/CrowTa00.htm">Tae Crowder</a></td><td class="left" data-stat="cfb"><a href="http://www.sports-reference.com/cfb/players/tae-crowder-1.html">College Stats</a></td><td class="right iz

('https://www.pro-football-reference.com/players/C/CostKJ00.htm', 'http://www.sports-reference.com/cfb/players/kj-costello-1.html')
('https://www.pro-football-reference.com/players/S/StevTo01.htm', 'http://www.sports-reference.com/cfb/players/tommy-stevens-1.html')
('https://www.pro-football-reference.com/players/B/BadaRi20.htm', 'http://www.sports-reference.com/cfb/players/richie-badar-1.html')
('https://www.pro-football-reference.com/players/B/BenkKu01.htm', 'http://www.sports-reference.com/cfb/players/kurt-benkert-1.html')
('https://www.pro-football-reference.com/players/P/PariBa00.htm', 'http://www.sports-reference.com/cfb/players/babe-parilli-1.html')
('https://www.pro-football-reference.com/players/B/BlouEd20.htm', 'http://www.sports-reference.com/cfb/players/ed-blount-1.html')
('https://www.pro-football-reference.com/players/S/SchwBo00.htm', 'http://www.sports-reference.com/cfb/players/bob-schweickert-1.html')
<tr><th class="left iz" data-stat="pos" scope="row"></th><td class="l

In [224]:
import os  
os.makedirs('data/', exist_ok=True)  
nfl.to_csv('data/nfl_data.csv')  
cfl.to_csv('data/cfl_data.csv')
names_s.to_csv('data/player_ids.csv')

In [335]:
def remove_other(s):
    n = re.sub('\D', '', str(s))
    if n != '':
        return int(n)
    return -1

def year_to_int(df):
    df_years = df['Year'].apply(remove_other)
    return df_years

def get_senior_years(df):
    l = []
    for id_ in df['player_id'].drop_duplicates():
        l.append(df[df['player_id'] == id_].iloc[-1])
    return l

In [336]:
cfl_int = cfl.copy()
nfl_int = nfl.copy()
cfl_int['Year'] = year_to_int(cfl_int)
nfl_int['Year'] = year_to_int(nfl_int)

In [337]:
cfl_rec = cfl_int[cfl_int['Year'] > 1990]
nfl_rec = nfl_int[nfl_int['Year'] > 1990]

In [338]:
cfl_seniors = pd.DataFrame(get_senior_years(cfl_rec))

In [343]:
cfl_seniors.describe()

,Year,G,Cmp,Att,Pct,Yds,Y/A,AY/A,TD,Int,Rate,player_id
count,382.000000,382.000000,382.000000,382.000000,380.000000,382.000000,380.000000,380.000000,382.000000,382.000000,380.000000,382.000000
mean,2006.062827,11.193717,213.146597,341.159686,60.519737,2748.641361,7.839474,7.547368,21.363874,8.471204,139.316842,388.633508
std,8.862095,2.629165,94.210600,138.070746,9.672808,1175.544087,1.553945,4.460151,11.243245,4.127618,36.734319,225.519492
min,1991.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-45.000000,0.000000,0.000000,-200.000000,2.000000
25%,1999.000000,11.000000,164.000000,278.250000,56.900000,2100.250000,7.100000,6.700000,15.000000,6.000000,127.375000,192.000000
50%,2006.000000,12.000000,220.000000,355.500000,61.550000,2863.500000,7.800000,7.900000,22.000000,8.000000,143.150000,382.500000
75%,2014.000000,13.000000,270.750000,429.750000,66.100000,3528.000000,8.700000,9.200000,29.000000,11.000000,157.350000,589.000000
max,2020.000000,15.000000,479.000000,712.000000,77.400000,5671.000000,11.600000,13.400000,60.000000,19.000000,206.900000,771.000000


In [349]:
nfl_rec

,Year,Age,Tm,Pos,No.,G,GS,QBrec,Cmp,Att,...,Sk%,NY/A,ANY/A,4QC,GWD,AV,player_id,1D,Awards,QBR
3,1991,26,DAL,NaN,7,8,4,4-0-0,68,137,...,4.2,6.01,6.08,2,2,3,1,NaN,NaN,NaN
4,1992,27,DAL,NaN,7,16,0,NaN,12,18,...,0,8.44,5.94,NaN,NaN,1,1,NaN,NaN,NaN
5,1993,28,PHO,QB,7,16,14,6-8-0,258,418,...,6.5,6.62,5.71,1,1,11,1,NaN,NaN,NaN
6,1994,29,ARI,qb,7,9,7,3-4-0,130,255,...,7.3,5.15,4.04,2,2,3,1,84,NaN,NaN
7,1995,30,JAX,qb,7,7,6,1-5-0,71,142,...,10.7,5.34,3.86,NaN,NaN,3,1,47,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,1996,40,MIN,QB,1.0,8,8,4-4-0,134,247,...,7.1,5.59,4.6,1.0,2.0,4,772,74.0,NaN,NaN
13,1997,41,SEA,QB,1.0,15,14,7-7-0,313,528,...,5.4,6.25,5.85,3.0,4.0,12,772,178.0,PB,NaN
14,1998,42,SEA,QB,1.0,10,10,4-6-0,145,258,...,7.9,5.33,4.83,NaN,NaN,5,772,79.0,NaN,NaN
15,1999,43,KAN,NaN,1.0,1,0,NaN,1,3,...,0.0,6.67,6.67,NaN,NaN,0,772,1.0,NaN,NaN
